In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
def make_box_image(img, s):
    m = s // 2
    h = img.shape[0]
    w = img.shape[1]
    img_ext = cv2.copyMakeBorder(img, m, m, m, m, cv2.BORDER_REFLECT_101, 0)
    ys = []
    for y in range(s):
        xs = []
        for x in range(s):
            xs.append(img_ext[y:y+h, x:x+w])
        ys.append(xs)
    return ys

In [ ]:
img_src = cv2.imread("Chrysanthemum_bayer_1024x768.pgm", cv2.IMREAD_ANYDEPTH).astype(np.int32)
raw = img_src

In [ ]:
def demosaic_g(raw):
    raws = make_box_image(raw, 5)

    a = np.abs(-raws[0][2] + 2*raws[2][2] - raws[4][2]) + np.abs(raws[1][2] - raws[3][2])
    b = np.abs(-raws[2][0] + 2*raws[2][2] - raws[2][4]) + np.abs(raws[2][1] - raws[2][3])

    g0 = (raws[1][2] + raws[3][2]) * 2 + (-raws[0][2] + 2*raws[2][2] - raws[4][2])
    g1 = (raws[2][1] + raws[2][3]) * 2 + (-raws[2][0] + 2*raws[2][2] - raws[2][4])

    g = np.where(a < b, g0*2, g1*2)
    g = np.where(a == b, g0+g1, g)
#    g = g0+g1 # test
    
    g = g // 8
    
    g = np.where(g <    0,    0, g)
    g = np.where(g > 1023, 1023, g)
    
    return g

In [ ]:
g = demosaic_g(raw)

In [ ]:
p0=0
p1=0
n0=1-p0
n1=1-p1

g[p0::2,p1::2] =   g[p0::2,p1::2]
g[p0::2,n1::2] = raw[p0::2,n1::2]
g[n0::2,p1::2] = raw[n0::2,p1::2]
g[n0::2,n1::2] =   g[n0::2,n1::2]

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
plt.imshow(raw)
plt.subplot(122)
plt.imshow(g)

In [ ]:
raws = make_box_image(raw, 5)
gs   = make_box_image(g, 5)

In [ ]:
a = np.abs(-gs[1][3] + 2*gs[2][2] - gs[3][1]) + np.abs(raws[1][3] - raws[3][1])
b = np.abs(-gs[1][1] + 2*gs[2][2] - gs[3][3]) + np.abs(raws[1][1] - raws[3][3])
print(a[2][0:10])
print(b[2][0:10])

In [ ]:
r = (raws[1][3] + raws[3][1]) * 2 + (-gs[1][3] + 2*gs[2][2] - gs[3][1])
l = (raws[1][1] + raws[3][3]) * 2 + (-gs[1][1] + 2*gs[2][2] - gs[3][3])
print(r[2][0:16])
print(l[2][0:16])

In [ ]:
x = np.where(a < b, r*2, l*2)
x = np.where(a == b, r+l, x)
#x = r+l # test
print(x[2][0:10]//8)

In [ ]:
h = (raws[2][1] + raws[2][3]) * 2 + (-gs[2][1] + 2*gs[2][2] - gs[2][3])
v = (raws[1][2] + raws[3][2]) * 2 + (-gs[1][2] + 2*gs[2][2] - gs[3][2])
print(h[2][0:10])
print(v[2][0:10])

In [ ]:
h = h // 4
v = v // 4
x = x // 8

In [ ]:
def img_clip(img):
    img = np.where(img <    0,    0, img)
    img = np.where(img > 1023, 1023, img)
    return img

In [ ]:
h = img_clip(h)
v = img_clip(v)
x = img_clip(x)

In [ ]:
print(raw[2][0:10])
print(g[2][0:10])
print(h[2][0:10])
print(v[2][0:10])
print(x[2][0:10])

In [ ]:
img = np.ndarray((3, raw.shape[0], raw.shape[1]), dtype=np.int32)

img[0][p0::2,p1::2] = raw[p0::2,p1::2]
img[0][p0::2,n1::2] =   h[p0::2,n1::2]
img[0][n0::2,p1::2] =   v[n0::2,p1::2]
img[0][n0::2,n1::2] =   x[n0::2,n1::2]

img[1][p0::2,p1::2] =   g[p0::2,p1::2]
img[1][p0::2,n1::2] = raw[p0::2,n1::2]
img[1][n0::2,p1::2] = raw[n0::2,p1::2]
img[1][n0::2,n1::2] =   g[n0::2,n1::2]

img[2][p0::2,p1::2] =   x[p0::2,p1::2]
img[2][p0::2,n1::2] =   v[p0::2,n1::2]
img[2][n0::2,p1::2] =   h[n0::2,p1::2]
img[2][n0::2,n1::2] = raw[n0::2,n1::2]

img = img.transpose(1, 2, 0)

plt.imshow(img)

In [ ]:
imgu8 = (img // 4).astype(np.uint8)
plt.figure(figsize=(8, 8))
plt.imshow(imgu8[:64,:64,0])

In [ ]:
imgu8 = (img // 4).astype(np.uint8)
plt.figure(figsize=(8, 8))
plt.imshow(imgu8[:64,:64,1])

In [ ]:
imgu8 = (img // 4).astype(np.uint8)
plt.figure(figsize=(8, 8))
plt.imshow(imgu8[:64,:64,2])

In [ ]:
cv2.imwrite("demosaic.png", imgu8[:,:,::-1])